In [1]:
import torch
from torch_geometric.explain import GNNExplainer
#from torch_geometric.explain import CFExplainer
from torch_geometric.nn import GCNConv
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
# Import other necessary modules as needed


In [2]:
!pip install torch_geometric
!pip install torch
import os
import torch
from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.nn import GCNConv, MessagePassing
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
import torch_geometric.transforms as T
import torch.nn.functional as F
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from torch import Tensor

%matplotlib inline

In [3]:
# !pip install jupyterthemes
# !jt -t chesterish
#!jt -r


In [4]:
%matplotlib inline

from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import TUDataset, Planetoid
from torch_geometric.nn import GCNConv, Set2Set
from torch_geometric.explain import GNNExplainer
import torch_geometric.transforms as T
import torch
import torch.nn.functional as F
import os
from tqdm import tqdm, trange

import matplotlib.pyplot as plt

In [5]:
from torch.nn import LogSoftmax
from torch_geometric.datasets import KarateClub
from torch_geometric.nn.conv import MessagePassing
import torch_geometric.transforms as T
from torch_geometric.datasets import ExplainerDataset
from torch_geometric.datasets.graph_generator import BAGraph
from sklearn.model_selection import train_test_split


# dataset = KarateClub()
# data = dataset[0]


dataset = ExplainerDataset(
    graph_generator=BAGraph(num_nodes=300, num_edges=5),
    motif_generator='house',
    num_motifs=80,
    transform=T.Constant(),
)
data = dataset[0]

idx = torch.arange(data.num_nodes)
train_idx, test_idx = train_test_split(idx, train_size=0.8, stratify=data.y)

def explain_message(self, inputs: Tensor, size_i: int) -> Tensor:
    """
    This is called when set_masks is activated (explain = True).
    """
    edge_mask = self._edge_mask

    if edge_mask is None:
        raise ValueError(f"Could not find a pre-defined 'edge_mask' as "
                            f"part of {self.__class__.__name__}.")

    if self._apply_sigmoid:
        edge_mask = edge_mask.sigmoid()

    # Some ops add self-loops to `edge_index`. We need to do the same for
    # `edge_mask` (but do not train these entries).
    if inputs.size(self.node_dim) != edge_mask.size(0):
        edge_mask = edge_mask[self._loop_mask]
        loop = edge_mask.new_ones(size_i)
        edge_mask = torch.cat([edge_mask, loop], dim=0)
    assert inputs.size(self.node_dim) == edge_mask.size(0)

    size = [1] * inputs.dim()
    size[self.node_dim] = -1
    hard_mask = (edge_mask.view(size) >= 0.5).float()

    return inputs * edge_mask.view(size) * hard_mask


class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 32)
        self.conv2 = GCNConv(32, dataset.num_classes)
        self.softmax = LogSoftmax(dim=1)

        # Add custom behavior to explain message
        self.conv1.explain_message = explain_message.__get__(self.conv1, MessagePassing)
        self.conv2.explain_message = explain_message.__get__(self.conv2, MessagePassing)


    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return self.softmax(x)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(0, 2000):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[train_idx], data.y[train_idx])
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        accuracy = sum(out.argmax(dim=-1)[test_idx] == data.y[test_idx]) / data.y[test_idx].shape[0]
        print(f"Test accuracy in epoch {epoch}: {accuracy}")

Test accuracy in epoch 0: 0.22142857313156128
Test accuracy in epoch 100: 0.41428571939468384
Test accuracy in epoch 200: 0.6571428775787354
Test accuracy in epoch 300: 0.7214285731315613
Test accuracy in epoch 400: 0.7071428298950195
Test accuracy in epoch 500: 0.7785714268684387
Test accuracy in epoch 600: 0.7357142567634583
Test accuracy in epoch 700: 0.7571428418159485
Test accuracy in epoch 800: 0.7785714268684387
Test accuracy in epoch 900: 0.7642857432365417
Test accuracy in epoch 1000: 0.7714285850524902
Test accuracy in epoch 1100: 0.7285714149475098
Test accuracy in epoch 1200: 0.7214285731315613
Test accuracy in epoch 1300: 0.7857142686843872
Test accuracy in epoch 1400: 0.7714285850524902
Test accuracy in epoch 1500: 0.7714285850524902
Test accuracy in epoch 1600: 0.7428571581840515
Test accuracy in epoch 1700: 0.7428571581840515
Test accuracy in epoch 1800: 0.8071428537368774
Test accuracy in epoch 1900: 0.7857142686843872


In [6]:
torch.argmax(model(data.x, data.edge_index)[83])
data.y[83]

tensor(0)

In [7]:
torch.tensor(3)*torch.tensor(3)

tensor(9)

In [8]:
for name, param in model.named_parameters():
    print(f"Parameter '{name}': requires_grad={param.requires_grad}")

Parameter 'conv1.bias': requires_grad=True
Parameter 'conv1.lin.weight': requires_grad=True
Parameter 'conv2.bias': requires_grad=True
Parameter 'conv2.lin.weight': requires_grad=True


In [10]:
from math import sqrt
from typing import Optional, Tuple, Union
import numpy as np
import torch
from torch import Tensor
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from torch_geometric.explain import ExplainerConfig, Explanation, ModelConfig
from torch_geometric.explain.algorithm import ExplainerAlgorithm
from torch_geometric.explain.algorithm.utils import clear_masks, set_masks
from torch_geometric.explain.config import MaskType, ModelMode, ModelTaskLevel
from torch_geometric.utils import to_dense_adj


class CFExplainer(ExplainerAlgorithm):
    r"""The CF-Explainer model from the `"CF-GNNExplainer: Counterfactual Explanations for Graph Neural
Networks"
    <https://arxiv.org/abs/2102.03322>`_ paper for generating CF explanations for GNNs:
    the minimal perturbation to the input (graph) data such that the prediction changes."""
    
    coeffs = {
        'edge_size': 0.1,
        'edge_reduction': 'sum',
        'node_feat_size': 1.0,
        'node_feat_reduction': 'mean',
        'edge_ent': 1.0,
        'node_feat_ent': 0.1,
        'EPS': 1e-15,
    }


    def __init__(self, epochs: int = 100, lr: float = 0.01, cf_optimizer = "SGD", n_momentum = 0, **kwargs):
        super().__init__()
        self.epochs = epochs
        self.lr = lr
        self.cf_optimizer = cf_optimizer
        self.n_momentum = n_momentum
        self.coeffs.update(kwargs)
        self.edge_mask = self.hard_edge_mask = None
        self.best_cf_example = None
        self.best_loss = np.inf

    def forward(self, model: torch.nn.Module, x: Tensor, edge_index: Tensor,
        *, index: int = None, **kwargs, ) -> Explanation:
        if isinstance(x, dict) or isinstance(edge_index, dict):
            raise ValueError(f"Heterogeneous graphs not yet supported in "
                             f"'{self.__class__.__name__}'")

        self._train(model, x, edge_index, index=index, **kwargs)

        if np.isinf(self.best_loss):
            raise Exception("No Counterfactual found.")

        edge_mask = self._post_process_mask(
            self.best_cf_example,
            apply_sigmoid=False,
        )
        self._clean_model(model)
        return Explanation(node_mask=None, edge_mask=edge_mask)

    def supports(self) -> bool:
        return True

    def _train(self, model: torch.nn.Module, x: Tensor, edge_index: Tensor,
        *, target: Tensor, index: int = None, **kwargs):

        # Set edge mask to all ones
        self._initialize_edge_mask(x, edge_index)
        parameters = []
        if self.edge_mask is not None:
            # This line sets masks in the Message passing module of PyG
            # as learnable Parameters and enables the call of "explain_message"
            set_masks(model, self.edge_mask, edge_index, apply_sigmoid=True)
            parameters.append(self.edge_mask)

        if self.cf_optimizer == "SGD" and self.n_momentum == 0.0:
            optimizer = torch.optim.SGD(parameters, lr=self.lr)
        elif self.cf_optimizer == "SGD" and self.n_momentum != 0.0:
            optimizer = torch.optim.SGD(parameters, lr=self.lr, nesterov=True, momentum=self.n_momentum)
        elif self.cf_optimizer == "Adadelta":
            optimizer = torch.optim.Adadelta(parameters, lr=self.lr)
        else:
            raise Exception("Optimizer is not currently supported.")


        for i in range(self.epochs):
            optimizer.zero_grad()
            y_hat, y = model(x, edge_index, **kwargs), target

            if index is not None:
                y_hat, y = y_hat[index], y[index]

            loss = self._loss(y_hat, y)
            loss.backward()
            optimizer.step()

            # Log best CF
            if (torch.argmax(y_hat) != y).float():
                if self.best_loss > loss:
                    binary_mask = (torch.sigmoid(self.edge_mask) > 0.5).int()
                    self.best_cf_example = torch.clone(1 - binary_mask)
                    self.best_loss = loss.item()

            # Log summary
            binary_mask = (torch.sigmoid(self.edge_mask) > 0.5).int()
            summary = f"Epoch {i} loss: {loss:.2f} | n_masked_edges: {sum(binary_mask == 0)}"
            summary = summary +  f"| pred_class: {torch.argmax(y_hat)}"
            summary = summary +  f"| max_grad: {self.edge_mask.grad.max():.2f}"
            print(summary)


    def _initialize_edge_mask(self, x: Tensor, edge_index: Tensor):
        edge_mask_type = self.explainer_config.edge_mask_type
        device = x.device
        E = edge_index.size(1)

        if edge_mask_type is None:
            self.edge_mask = None
        elif edge_mask_type == MaskType.object:
            self.edge_mask = Parameter(torch.ones(E, device=device))
        else:
            assert False

    def _loss(self, y_hat: Tensor, y: Tensor) -> Tensor:
        if self.model_config.mode == ModelMode.binary_classification:
            loss = self._loss_binary_classification(y_hat, y)
        elif self.model_config.mode == ModelMode.multiclass_classification:
            loss = self._loss_multiclass_classification(y_hat, y)
        elif self.model_config.mode == ModelMode.regression:
            loss = self._loss_regression(y_hat, y)
        else:
            assert False

        indicator = (torch.argmax(y_hat) == y).float()
        dist_loss_l1 = sum(abs(self.edge_mask - torch.ones_like(self.edge_mask)))
        loss = - indicator * loss + self.coeffs['edge_size'] * dist_loss_l1
        return loss

    def _clean_model(self, model):
        clear_masks(model)
        self.edge_mask = None

In [20]:
explainer = Explainer(
    model=model,
    algorithm=CFExplainer(epochs=100, lr=.25),
    explanation_type='model',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='graph',
        return_type='log_probs',
    ),
)

node_index = 100
explanation = explainer(data.x, data.edge_index, index=node_index)
path = 'subgraph.pdf'
explanation.visualize_graph(path)

print('Ground-truth label', data.y[node_index].item())
print(f"Counterfactual visualization plot has been saved to '{path}'")

# Analyze mask
mask = explanation.edge_mask > 0.5
print(f"Masked edges for node {node_index}: \n", data.edge_index[:,  mask])

Epoch 0 loss: -0.77 | n_masked_edges: 0| pred_class: 2| max_grad: 0.03
Epoch 1 loss: -0.76 | n_masked_edges: 0| pred_class: 2| max_grad: 0.09
Epoch 2 loss: -0.57 | n_masked_edges: 0| pred_class: 2| max_grad: 0.13
Epoch 3 loss: -0.72 | n_masked_edges: 0| pred_class: 2| max_grad: 0.09
Epoch 4 loss: -0.66 | n_masked_edges: 0| pred_class: 2| max_grad: 0.12
Epoch 5 loss: -0.69 | n_masked_edges: 0| pred_class: 2| max_grad: 0.14
Epoch 6 loss: -0.68 | n_masked_edges: 0| pred_class: 2| max_grad: 0.11
Epoch 7 loss: -0.72 | n_masked_edges: 0| pred_class: 2| max_grad: 0.12
Epoch 8 loss: -0.67 | n_masked_edges: 0| pred_class: 2| max_grad: 0.14
Epoch 9 loss: -0.75 | n_masked_edges: 0| pred_class: 2| max_grad: 0.12
Epoch 10 loss: -0.68 | n_masked_edges: 0| pred_class: 2| max_grad: 0.11
Epoch 11 loss: -0.76 | n_masked_edges: 0| pred_class: 2| max_grad: 0.14
Epoch 12 loss: -0.70 | n_masked_edges: 0| pred_class: 2| max_grad: 0.12
Epoch 13 loss: 0.20 | n_masked_edges: 0| pred_class: 0| max_grad: 0.10
Epo

In [17]:
for i in explanation['edge_mask']:
    if i > 0:
        print(i)

In [38]:
from math import sqrt
from typing import Optional, Tuple, Union

import torch
from torch import Tensor
from torch.nn.parameter import Parameter

from torch_geometric.explain import ExplainerConfig, Explanation, ModelConfig
from torch_geometric.explain.algorithm import ExplainerAlgorithm
from torch_geometric.explain.algorithm.utils import clear_masks, set_masks
from torch_geometric.explain.config import MaskType, ModelMode, ModelTaskLevel


class GNNExplainer(ExplainerAlgorithm):
    coeffs = {
        'edge_size': 0.005,
        'edge_reduction': 'sum',
        'node_feat_size': 1.0,
        'node_feat_reduction': 'mean',
        'edge_ent': 1.0,
        'node_feat_ent': 0.1,
        'EPS': 1e-15,
    }

    def __init__(self, epochs: int = 100, lr: float = 0.01, **kwargs):
        super().__init__()
        self.epochs = epochs
        self.lr = lr
        self.coeffs.update(kwargs)

        self.node_mask = self.hard_node_mask = None
        self.edge_mask = self.hard_edge_mask = None

    def forward(
        self,
        model: torch.nn.Module,
        x: Tensor,
        edge_index: Tensor,
        *,
        target: Tensor,
        index: Optional[Union[int, Tensor]] = None,
        **kwargs,
    ) -> Explanation:
        if isinstance(x, dict) or isinstance(edge_index, dict):
            raise ValueError(f"Heterogeneous graphs not yet supported in "
                             f"'{self.__class__.__name__}'")

        self._train(model, x, edge_index, target=target, index=index, **kwargs)

        node_mask = self._post_process_mask(
            self.node_mask,
            self.hard_node_mask,
            apply_sigmoid=True,
        )
        edge_mask = self._post_process_mask(
            self.edge_mask,
            self.hard_edge_mask,
            apply_sigmoid=True,
        )

        self._clean_model(model)

        return Explanation(node_mask=node_mask, edge_mask=edge_mask)

    def supports(self) -> bool:
        return True

    def _train(
        self,
        model: torch.nn.Module,
        x: Tensor,
        edge_index: Tensor,
        *,
        target: Tensor,
        index: Optional[Union[int, Tensor]] = None,
        **kwargs,
    ):
        self._initialize_masks(x, edge_index)

        parameters = []
        if self.node_mask is not None:
            parameters.append(self.node_mask)
        if self.edge_mask is not None:
            set_masks(model, self.edge_mask, edge_index, apply_sigmoid=True)
            parameters.append(self.edge_mask)

        optimizer = torch.optim.Adam(parameters, lr=self.lr)

        for i in range(self.epochs):
            optimizer.zero_grad()

            h = x if self.node_mask is None else x * self.node_mask.sigmoid()
            y_hat, y = model(h, edge_index, **kwargs), target

            if index is not None:
                y_hat, y = y_hat[index], y[index]

            loss = self._loss(y_hat, y)

            loss.backward()
            optimizer.step()

            # In the first iteration, we collect the nodes and edges that are
            # involved into making the prediction. These are all the nodes and
            # edges with gradient != 0 (without regularization applied).
            if i == 0 and self.node_mask is not None:
                if self.node_mask.grad is None:
                    raise ValueError("Could not compute gradients for node "
                                     "features. Please make sure that node "
                                     "features are used inside the model or "
                                     "disable it via `node_mask_type=None`.")
                self.hard_node_mask = self.node_mask.grad != 0.0
            if i == 0 and self.edge_mask is not None:
                if self.edge_mask.grad is None:
                    raise ValueError("Could not compute gradients for edges. "
                                     "Please make sure that edges are used "
                                     "via message passing inside the model or "
                                     "disable it via `edge_mask_type=None`.")
                self.hard_edge_mask = self.edge_mask.grad != 0.0

    def _initialize_masks(self, x: Tensor, edge_index: Tensor):
        node_mask_type = None
        edge_mask_type = self.explainer_config.edge_mask_type

        device = x.device
        (N, F), E = x.size(), edge_index.size(1)

        std = 0.1
        if node_mask_type is None:
            self.node_mask = None
        elif node_mask_type == MaskType.object:
            self.node_mask = Parameter(torch.randn(N, 1, device=device) * std)
        elif node_mask_type == MaskType.attributes:
            self.node_mask = Parameter(torch.randn(N, F, device=device) * std)
        elif node_mask_type == MaskType.common_attributes:
            self.node_mask = Parameter(torch.randn(1, F, device=device) * std)
        else:
            assert False

        if edge_mask_type is None:
            self.edge_mask = None
        elif edge_mask_type == MaskType.object:
            std = torch.nn.init.calculate_gain('relu') * sqrt(2.0 / (2 * N))
            self.edge_mask = Parameter(torch.randn(E, device=device) * std)
        else:
            assert False

    def _loss(self, y_hat: Tensor, y: Tensor) -> Tensor:
        if self.model_config.mode == ModelMode.binary_classification:
            loss = self._loss_binary_classification(y_hat, y)
        elif self.model_config.mode == ModelMode.multiclass_classification:
            loss = self._loss_multiclass_classification(y_hat, y)
        elif self.model_config.mode == ModelMode.regression:
            loss = self._loss_regression(y_hat, y)
        else:
            assert False

        if self.hard_edge_mask is not None:
            assert self.edge_mask is not None
            m = self.edge_mask[self.hard_edge_mask].sigmoid()
            edge_reduce = getattr(torch, self.coeffs['edge_reduction'])
            loss = loss + self.coeffs['edge_size'] * edge_reduce(m)
            ent = -m * torch.log(m + self.coeffs['EPS']) - (
                1 - m) * torch.log(1 - m + self.coeffs['EPS'])
            loss = loss + self.coeffs['edge_ent'] * ent.mean()

        if self.hard_node_mask is not None:
            assert self.node_mask is not None
            m = self.node_mask[self.hard_node_mask].sigmoid()
            node_reduce = getattr(torch, self.coeffs['node_feat_reduction'])
            loss = loss + self.coeffs['node_feat_size'] * node_reduce(m)
            ent = -m * torch.log(m + self.coeffs['EPS']) - (
                1 - m) * torch.log(1 - m + self.coeffs['EPS'])
            loss = loss + self.coeffs['node_feat_ent'] * ent.mean()
        print(loss)
        return loss

    def _clean_model(self, model):
        clear_masks(model)
        self.node_mask = self.hard_node_mask = None
        self.edge_mask = self.hard_edge_mask = None

In [34]:
from math import sqrt
from typing import Optional, Tuple, Union

import numpy as np
import torch
from torch import Tensor
from torch.nn.parameter import Parameter
import torch.nn.functional as F


from torch_geometric.explain import ExplainerConfig, Explanation, ModelConfig
from torch_geometric.explain.algorithm import ExplainerAlgorithm
from torch_geometric.explain.algorithm.utils import clear_masks, set_masks
from torch_geometric.explain.config import MaskType, ModelMode, ModelTaskLevel
from torch_geometric.utils import to_dense_adj
#from torch_geometric.utils import dense_adjacency


class CFExplainer(ExplainerAlgorithm):
    r"""The CF-Explainer model from the `"CF-GNNExplainer: Counterfactual Explanations for Graph Neural
Networks"
    <https://arxiv.org/abs/2102.03322>`_ paper for generating CF explanations for GNNs: 
    the minimal perturbation to the input (graph) data such that the prediction changes.

    .. note::

        For an example of using :class:`GNNExplainer`, see
        `examples/explain/gnn_explainer.py <https://github.com/pyg-team/
        pytorch_geometric/blob/master/examples/explain/gnn_explainer.py>`_,
        `examples/explain/gnn_explainer_ba_shapes.py <https://github.com/
        pyg-team/pytorch_geometric/blob/master/examples/
        explain/gnn_explainer_ba_shapes.py>`_, and `examples/explain/
        gnn_explainer_link_pred.py <https://github.com/pyg-team/
        pytorch_geometric/blob/master/examples/explain/gnn_explainer_link_pred.py>`_.

    Args:
        epochs (int, optional): The number of epochs to train.
            (default: :obj:`100`)
        lr (float, optional): The learning rate to apply.
            (default: :obj:`0.01`)
        **kwargs (optional): Additional hyper-parameters to override default
            settings in
            :attr:`~torch_geometric.explain.algorithm.GNNExplainer.coeffs`.
    """

    coeffs = {
        'edge_size': 0.005,
        'edge_reduction': 'sum',
        'beta' : .001,
        'node_feat_size': 1.0,
        'node_feat_reduction': 'mean',
        'edge_ent': 1.0,
        'node_feat_ent': 0.1,
        'EPS': 1e-15,
    }

    def __init__(self, epochs: int = 100, lr: float = 0.01, cf_optimizer = "SGD", n_momentum = 0, **kwargs):
        super().__init__()
        self.epochs = epochs
        self.lr = lr
        self.cf_optimizer = cf_optimizer
        self.n_momentum = n_momentum
        self.coeffs.update(kwargs)
        self.node_mask = self.hard_node_mask = None
        self.edge_mask = self.hard_edge_mask = None
        self.best_cf_example = None
        self.best_loss = np.inf
        
    def forward(
        self,
        model: torch.nn.Module,
        x: Tensor,
        edge_index: Tensor,
        *,
        target: Tensor,
        index: Optional[Union[int, Tensor]] = None,
        **kwargs,
    ) -> Explanation:
        if isinstance(x, dict) or isinstance(edge_index, dict):
            raise ValueError(f"Heterogeneous graphs not yet supported in "
                             f"'{self.__class__.__name__}'")

        self._train(model, x, edge_index, target=target, index=index, **kwargs)

        # node_mask = self._post_process_mask(
        #     self.best_cf_example[0],
        #     self.hard_node_mask,
        #     apply_sigmoid=True,
        # )
        edge_mask = self._post_process_mask(
            self.best_cf_example,
            self.hard_edge_mask,
            apply_sigmoid=True,
        )

        self._clean_model(model)

        return Explanation(node_mask=node_mask, edge_mask=edge_mask)

    def supports(self) -> bool:
        return True
    
    def _train(
        self,
        model: torch.nn.Module,
        x: Tensor,
        edge_index: Tensor,
        *,
        target: Tensor,
        index: Optional[Union[int, Tensor]] = None,
        **kwargs,
    ):
        self._initialize_masks(x, edge_index)

        parameters = []
        if self.node_mask is not None:
            parameters.append(self.node_mask)
        if self.edge_mask is not None:
            set_masks(model, self.edge_mask, edge_index, apply_sigmoid=True)
            parameters.append(self.edge_mask)
    
        if self.cf_optimizer == "SGD" and self.n_momentum == 0.0:
            optimizer = torch.optim.SGD(parameters, lr=self.lr)
        elif self.cf_optimizer == "SGD" and self.n_momentum != 0.0:
            optimizer = torch.optim.SGD(parameters, lr=self.lr, nesterov=True, momentum=n_momentum)
        elif self.cf_optimizer == "Adadelta":
            optimizer = torch.optim.Adadelta(parameters, lr=self.lr)
        else:
            raise Exception("Optimizer is not currently supported.")
        
        num_cf_examples = 0
        original_prediction  = model(x, edge_index, **kwargs)
        for i in range(self.epochs):
            optimizer.zero_grad()
            h = x if self.node_mask is None else x * self.node_mask.sigmoid()
            discrete_edge_mask = torch.where(torch.sigmoid(self.edge_mask)>=0.5, 1, 0)
            set_masks(model, discrete_edge_mask, edge_index, apply_sigmoid=False)
            y_hat, y = model(h, edge_index, **kwargs), original_prediction
            y_hat_discrete, y_discrete = y_hat.argmax(dim=1), y.argmax(dim=1)
            set_masks(model, self.edge_mask, edge_index, apply_sigmoid=True)

            if index is not None:
                y_hat, y = y_hat[index], y[index]

            loss = self._loss(y_hat, y, edge_index)

            if loss.item() < best_loss:
                best_loss = loss
                self.best_cf_example = self.edge_mask
            
            loss.backward()
            optimizer.step()

            # In the first iteration, we collect the nodes and edges that are
            # involved into making the prediction. These are all the nodes and
            # edges with gradient != 0 (without regularization applied).
            if i == 0 and self.node_mask is not None:
                if self.node_mask.grad is None:
                    raise ValueError("Could not compute gradients for node "
                                     "features. Please make sure that node "
                                     "features are used inside the model or "
                                     "disable it via `node_mask_type=None`.")
                self.hard_node_mask = self.node_mask.grad != 0.0
            if i == 0 and self.edge_mask is not None:
                if self.edge_mask.grad is None:
                    raise ValueError("Could not compute gradients for edges. "
                                     "Please make sure that edges are used "
                                     "via message passing inside the model or "
                                     "disable it via `edge_mask_type=None`.")
                self.hard_edge_mask = self.edge_mask.grad != 0.0

    def _initialize_masks(self, x: Tensor, edge_index: Tensor):
        node_mask_type = self.explainer_config.node_mask_type
        edge_mask_type = self.explainer_config.edge_mask_type

        device = x.device
        (N, F), E = x.size(), edge_index.size(1)

        std = 0.1
        if node_mask_type is None:
            self.node_mask = None
        elif node_mask_type == MaskType.object:
            self.node_mask = Parameter(torch.randn(N, 1, device=device) * std)
        elif node_mask_type == MaskType.attributes:
            self.node_mask = Parameter(torch.randn(N, F, device=device) * std)
        elif node_mask_type == MaskType.common_attributes:
            self.node_mask = Parameter(torch.randn(1, F, device=device) * std)
        else:
            assert False

        if edge_mask_type is None:
            self.edge_mask = None
        elif edge_mask_type == MaskType.object:
            std = torch.nn.init.calculate_gain('relu') * sqrt(2.0 / (2 * N))
            self.edge_mask = Parameter(torch.randn(E, device=device) * std)
        else:
            assert False

    # def _initialize_masks(self, x: Tensor, edge_index: Tensor):
    #     node_mask_type = self.explainer_config.node_mask_type
    #     edge_mask_type = self.explainer_config.edge_mask_type

    #     device = x.device
    #     (N, F), E = x.size(), edge_index.size(1)

    #     if node_mask_type is None:
    #         self.node_mask = None
    #     elif node_mask_type == MaskType.object:
    #         self.node_mask = Parameter(torch.ones(N, 1, device=device))
    #     elif node_mask_type == MaskType.attributes:
    #         self.node_mask = Parameter(torch.ones(N, F, device=device))
    #     elif node_mask_type == MaskType.common_attributes:
    #         self.node_mask = Parameter(torch.ones(1, F, device=device))
    #     else:
    #         assert False


    #     if edge_mask_type is None:
    #         self.edge_mask = None
    #     elif edge_mask_type == MaskType.object:
    #         self.edge_mask = Parameter(torch.ones(E, device=device))
    #     else:
    #         assert False


    def _loss(self, y_hat: Tensor, y: Tensor, edge_index) -> Tensor:
        y_hat_discrete = y_hat.argmax(dim=1)
        y_discrete = y.argmax(dim=1)

        pred_same = (y_hat_discrete == y_discrete).float()
        
        # if self.model_config.mode == ModelMode.binary_classification:
        #     loss = self._loss_binary_classification(y_hat, y)
        # elif self.model_config.mode == ModelMode.multiclass_classification:
        #     loss = self._loss_multiclass_classification(y_hat, y)
        # elif self.model_config.mode == ModelMode.regression:
        #     loss = self._loss_regression(y_hat, y)
        # else:
        #     assert False
        # Want negative in front to maximize loss instead of minimizing it to find CFs
        discrete_edge_mask = torch.where(torch.sigmoid(self.edge_mask)>=0, 1, 0)

        loss_pred = - F.nll_loss(y_hat, y_discrete)
        adj = dense_adjacency(edge_index, edge_attr=None, num_nodes=None)
        discrete_adj = torch.where(torch.sigmoid(self.edge_mask) >= 0, 1, 0)
        loss_graph_dist = torch.sum(torch.abs(adj - discrete_adj)) / 2

        #loss_graph_dist = sum(sum(abs(to_dense_adj(edge_index) - to_dense_adj(discrete_edge_mask)))) / 2      # Number of edges changed (symmetrical)

		# Zero-out loss_pred with pred_same if prediction flips
        loss_total = pred_same * loss_pred + self.coeff['beta'] * loss_graph_dist


        # if self.hard_edge_mask is not None:
        #     assert self.edge_mask is not None
        #     m = self.edge_mask[self.hard_edge_mask].sigmoid()
        #     edge_reduce = getattr(torch, self.coeffs['edge_reduction'])
        #     loss = loss + self.coeffs['edge_size'] * edge_reduce(m)
        #     ent = -m * torch.log(m + self.coeffs['EPS']) - (
        #         1 - m) * torch.log(1 - m + self.coeffs['EPS'])
        #     loss = loss + self.coeffs['edge_ent'] * ent.mean()

        # if self.hard_node_mask is not None:
        #     assert self.node_mask is not None
        #     m = self.node_mask[self.hard_node_mask].sigmoid()
        #     node_reduce = getattr(torch, self.coeffs['node_feat_reduction'])
        #     loss16 = loss + self.coeffs['node_feat_size'] * node_reduce(m)
        #     ent = -m * torch.log(m + self.coeffs['EPS']) - (
        #         1 - m) * torch.log(1 - m + self.coeffs['EPS'])
        #     loss = loss + self.coeffs['node_feat_ent'] * ent.mean()

        return loss_total

    def _clean_model(self, model):
        clear_masks(model)
        self.node_mask = self.hard_node_mask = None
        self.edge_mask = self.hard_edge_mask = None
